### EverPost: create Evernote notes with Mercury API and Python.

By Anzhelika Boiko

email angelika.boyko@gmail.com 

2017FA-CIS-024C-101

#### Final Project 

Evernote notes from a URL or text file with as
few clicks as possible. EverPost uses Mercury API to
create clean, nice-looking and easy to read notes.

Before using the script you need to obtain valid Mercury and
Evernote API tokens. 
You can create your own, or you can use those provided with the script.

#### Evernote API
https://dev.evernote.com/doc/start/python.php

#### Grab Mercury API keys from:

https://mercury.postlight.com/web-parser/

#### Grab Evernote Cloud API token from:

https://sandbox.evernote.com/api/DeveloperToken.action

#### Dependencies:

To convert HTML into ENLM  (the markup used by Evernote, see:
http://dev.evernote.com/start/core/enml.php) which permitts a limited
subset of XHTML tags, EverPost uses the asciinator aka html2text.py
that can be downloaded from:

http://www.aaronsw.com/2002/html2text/

#### Configuration:

Replace R_TOKEN and EN_TOKEN values (if you use your own tokens) in EverPost with the tokens  obtained
from Mercury and Evernote sites.

#### Installation

pip install markdown2

pip install evernote

pip install html2text

#### The result will be posted as a note on Evernote account.

This is sandbox development account, feel free to use my password and login:

https://sandbox.evernote.com/Home.action?#n=a0bf87ea-b6a7-44e7-8245-a14ad2d1a24e&s=s1&ses=4&sh=2&sds=5&

password angelika.boyko@gmail.com

login wMu-LBn-e6F-yUb

In [3]:
import requests
import re
import markdown2
from string import Template
from bs4 import BeautifulSoup
import sys
import cgi
import html2text
import evernote.edam.userstore.constants as UserStoreConstants
import evernote.edam.type.ttypes as Types
from evernote.api.client import EvernoteClient

R_SRV_URL = "https://mercury.postlight.com/parser"

# Replace the values below with the values obtained from Mercury
# and Evernote dev sites, see notes above
R_TOKEN = "uA6d56LnDEfFwpbUBOKMxTgBbUWq2hixkxL68sVk"
EN_TOKEN = "S=s1:U=943d5:E=167999f5445:C=16041ee27d8:P=1cd:A=en-devtoken:V=2:H=cc60790882690e980a312ec029b2fca3"

hdr_tmpl = Template("""\
# $title \n
**Author:** $author \n
**URL:** [$url]($url) \n\n""")


class MercuryRequest(object):
    def __init__(self, token, r_server):
        self.token = token
        self.server = r_server

    def make_request(self, url):
        # Get parsed html from Mercury
        pars = dict()
        hdrs = dict()
        hdrs["x-api-key"] = self.token
        pars["url"] = url
        r = requests.get(self.server, params=pars, headers=hdrs)
        title = r.json()["title"]
        html = r.json()["content"]
        url = r.json()["url"]
        author = r.json()["author"]
        return title, html, url, author


class EvernotePoster(object):
    """Post notes to evernote"""
    def __init__(self, token):
        self.token = token
        self.client = EvernoteClient(token=EN_TOKEN, sandbox=True)
        self.user_store = self.client.get_user_store()
        self.note_store = self.client.get_note_store()

    def check_version(self):
        """Check for version"""
        version_ok = self.user_store.checkVersion(
            "Version check",
            UserStoreConstants.EDAM_VERSION_MAJOR,
            UserStoreConstants.EDAM_VERSION_MINOR
        )
        if not version_ok:
            return -1
        else:
            return 0

    def _make_note(self, title, content, url):
        """Prepare a note to be posted"""
        note = Types.Note()
        note.title = title
        # Set up note attributes
        attrs = Types.NoteAttributes()
        attrs.sourceURL = url
        note.attributes = attrs
        note.content = '<?xml version="1.0" encoding="UTF-8"?>'
        note.content += '<!DOCTYPE en-note SYSTEM '  \
                        '"http://xml.evernote.com/pub/enml2.dtd">'
        # Wrap content in <en-note>
        note.content += "<en-note>"
        note.content += content
        note.content += "</en-note>"
        return note

    def post_note(self, title, note, url):
        """Post a note to Evernote"""
        ver = self.check_version()
        if ver < 0:
            print "*** VERSION ERROR: Update client to the latest version."
        else:
            note = self._make_note(title, note, url)
            created_note = self.note_store.createNote(note)
        return created_note.guid


class ContentCreator(object):
    """Create note content from URL"""
    def __init__(self, clip):
        self.clip = clip

    def make_content(self):
        """Use Mercury parser to get URL content"""
        if self.is_valid_url(self.clip):
            rr = MercuryRequest(R_TOKEN, R_SRV_URL)
            title, html, url, author = rr.make_request(self.clip)
            # Create preamble with title and author
            preamble = hdr_tmpl.substitute(title=title,
                                           author=author,
                                           url=url)
            try:
                txt = preamble + html2text.html2text(html)
            except:
                txt = "<p>This site uses weird formatting not supported by html2text. Cannot convert.</p>"
            enml = self.html2enml(markdown2.markdown(txt))
        else:
            print "You entered invalid URL. Exiting."
            sys.exit(-2)
        # We need to ensure that unicode is processed properly
        return title.encode("utf-8"), enml.encode("utf-8"), url

    def is_valid_url(self, url):
        """Validate URL, return False if not"""
        # Borrowed from Django
        regex = re.compile(
            r"^https?://"  # http:// or https://
            r"(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|"  # domain...
            r"localhost|"  # localhost...
            r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})"  # ...or ip
            r"(?::\d+)?"  # optional port
            r"(?:/?|[/?]\S+)$", re.IGNORECASE)
        return url is not None and regex.search(url)

    @staticmethod
    def html2enml(html):
        """Convert to ENLM and make links look better."""
        soup = BeautifulSoup(html, "html.parser")
        links = soup.find_all("a")
        for link in links:
            if "javascript" in link["href"] or link["href"] == "":
                link["href"] = "#"
            else:
                link["style"] = "text-decoration:none;color:#4682B4;"

        # ENML does not allow html and body
        try:
           out = soup.body.prettify().replace("body", "div")
        except AttributeError:
            out = "<div>" + soup.prettify() + "</div>"
        return out


if __name__ == "__main__":
    url = raw_input("Enter a valid URL: ")

    mr = MercuryRequest(R_TOKEN, R_SRV_URL)
    try:
        t, h, u, a = mr.make_request(url)
    except KeyError:
        print "*** Error: Cannot retrieve title. Are you entered a valid URL?"

    cc = ContentCreator(url)
    title, enml, url = cc.make_content()
    
    # Make sure we have something to post
    if enml is not None:
        ep = EvernotePoster(EN_TOKEN)
        guid = ep.post_note(title, enml, url)
        if guid is not None:
            print "=== Success!. Posted note with GUID: ", guid
        else:
            print "*** ERROR: Cannot post to Evernote."
    else:
        print "*** ERROR: Nothing to post."

Enter a valid URL: http://www.apt2bbakingco.com/home/a-sweet-feast-paris-food-photography-workshop-with-olaiya-land-2018
=== Success!. Posted note with GUID:  47b4bcbd-3331-41cd-b02a-92fcdf14da8c
